In [1]:
import compute_convergence
import data
import glob

In [7]:
d = data.load_data("../output/synthetic/s1/")
d = data.postprocess(d)

In [8]:
d.to_csv("s1_data.csv")

In [4]:
import pandas as pd
d = pd.read_csv("s1_data.csv")

In [24]:
ll1= d[(d.crit == "llratio") & (d.th == 1)]
ll1.to_csv("ll1_data.csv")